# 垃圾识别  Garbage identify

### 导入头文件 Import head file

In [1]:
#!/usr/bin/env python
# coding: utf-8
import Arm_Lib
import os
import cv2 as cv
import threading
from time import sleep
import ipywidgets as widgets
from IPython.display import display
from dofbot_utils.fps import FPS
from ultralytics import YOLO
from dofbot_utils.robot_controller import Robot_Controller

/usr/local/lib/python3.10/dist-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


### 创建实例,初始化参数 Create the instance and initialize the parameters

In [2]:
robot = Robot_Controller()
robot.move_look_map()

In [3]:
yolo_model = YOLO("./garbage.engine", task='detect')
fps = FPS()
model = "General"

### 创建控件 Creating widget

In [4]:
button_layout      = widgets.Layout(width='320px', height='60px', align_self='center')
output = widgets.Output()
# 退出
exit_button = widgets.Button(description='Exit', button_style='danger', layout=button_layout)
imgbox = widgets.Image(format='jpg', height=480, width=640, layout=widgets.Layout(align_self='center'))
controls_box = widgets.VBox([imgbox, exit_button], layout=widgets.Layout(align_self='center'))

### 抓取控制  grab control

In [5]:
def exit_button_Callback(value):
    global model
    model = 'Exit'
    with output: print(model)
exit_button.on_click(exit_button_Callback)

### 主程序 Main process

In [6]:
def camera():
    # 打开摄像头 Open camera
    capture = cv.VideoCapture(0)
    capture.set(cv.CAP_PROP_FRAME_WIDTH, 640)
    capture.set(cv.CAP_PROP_FRAME_HEIGHT, 480)
    # 当摄像头正常打开的情况下循环执行
    while capture.isOpened():
        try:
            _, img = capture.read()
            fps.update_fps()
            results = yolo_model(img, save=False, verbose=False)  # 使用YOLO11进行物体检测
            annotated_frame = results[0].plot(
                labels = True, # 显示标签
                conf = False,  # 显示置信度
                boxes = True,  # 绘制边界框
            )
            if model == 'Exit':
                cv.destroyAllWindows()
                capture.release()
                break
            fps.show_fps(annotated_frame)
            imgbox.value = cv.imencode('.jpg', annotated_frame)[1].tobytes()
        except Exception as e:
            capture.release()
            print(e)
            break

### 启动  Start

In [7]:
# Please place the building block in the center of the cross (the picture is facing the mechanical arm)
# 请将积木块正放在十字中心(图片正对机械臂)
display(controls_box,output)
threading.Thread(target=camera, ).start()


Output()

[ WARN:0@8.282] global cap_gstreamer.cpp:1777 open OpenCV | GStreamer warning: Cannot query video position: status=0, value=-1, duration=-1


Loading garbage.engine for TensorRT inference...
[04/21/2025-10:51:19] [TRT] [I] Loaded engine size: 11 MiB
[04/21/2025-10:51:19] [TRT] [W] Using an engine plan file across different models of devices is not recommended and is likely to affect performance or even cause errors.
[04/21/2025-10:51:19] [TRT] [I] [MemUsageChange] TensorRT-managed allocation in IExecutionContext creation: CPU +0, GPU +19, now: CPU 0, GPU 28 (MiB)
